# Homework: Decipherment

In [2]:
from collections import defaultdict, Counter
import ngram
from ngram import *
import collections
import pprint
import math
import bz2
import numpy
import time
import pandas as pd
import numpy as np
pp = pprint.PrettyPrinter(width=45, compact=True)


In [3]:
def read_file(filename):
    if filename[-4:] == ".bz2":
        with bz2.open(filename, 'rt') as f:
            content = f.read()
            f.close()
    else:
        with open(filename, 'r') as f:
            content = f.read()
            f.close()
    return content

def get_statistics(content, cipher=True):
    stats = {}
    content = list(content)
    split_content = [x for x in content if x != '\n' and x!=' ']
    length = len(split_content)
    symbols = set(split_content)
    uniq_sym = len(list(symbols))
    freq = collections.Counter(split_content)
    rel_freq = {}
    for sym, frequency in freq.items():
        rel_freq[sym] = (frequency/length)*100
        
    if cipher:
        stats = {'content':split_content, 'length':length, 'vocab':list(symbols), 'vocab_length':uniq_sym, 'frequencies':freq, 'relative_freq':rel_freq}
    else:
        stats = {'length':length, 'vocab':list(symbols), 'vocab_length':uniq_sym, 'frequencies':freq, 'relative_freq':rel_freq}
    return stats

def find_mappings(ciphertext, plaintext):
    mappings = defaultdict(dict)
    hypotheses = defaultdict(dict)
    
    for symbol in ciphertext['vocab']:
        for letter in plaintext['vocab']:
            hypotheses[symbol][letter] = abs(math.log((ciphertext['relative_freq'][symbol]/plaintext['relative_freq'][letter])))
    
    for sym in hypotheses.keys():
        winner = sorted(hypotheses[sym].items(), key=lambda kv: kv[1])
        mappings[sym] = winner[1][0]
    
    return mappings

In [4]:
cipher = read_file("data/cipher.txt")
plaintxt = read_file("data/default.wiki.txt.bz2")


cipher_desc = get_statistics(cipher, cipher=True)
plaintxt_desc = get_statistics(plaintxt, cipher=False)

mapping = find_mappings(cipher_desc, plaintxt_desc)

english_text = []
for symbol in cipher_desc['content']:
    english_text.append(mapping[symbol])
decipherment = ('').join(english_text)
#print(decipherment)
print(cipher_desc['vocab'])

['B', 'G', '∏', '^', 'y', 'À', 'Ç', '‘', 'ƒ', '∆', 'Z', 'u', '–', '£', '∑', 'I', 'Ã', 'T', 'V', '“', 'X', '+', 'µ', 'M', '\\', 'j', 'L', 'Ω', '√', 'J', '∞', 'W', 'O', 'F', 'N', '≈', 'H', 'º', '¢', 'E', '•', '∫', 'Q', 'π', '—', '§', 'æ', '/', 'S', 'K', 'A', 'D', 'P', 'R']


In [5]:
symbol_list = []; 
symbol_relFreq = []; 
for x, y in cipher_desc["relative_freq"].items():
    symbol_list.append(x)
    symbol_relFreq.append(y)
    
index_names = {}
for i in range(54):
    index_names[i] = symbol_list[i]
    
test_data = numpy.ones((54,26))
df = pd.DataFrame(test_data, columns = plaintxt_desc['vocab'])
df=df.rename(index = index_names )


## LOADING NGRAM MODEL

In [6]:
freq_dict=[ (k,v) for k,v in zip(cipher_desc['frequencies'].keys(),cipher_desc['frequencies'].values())]
sorted_freq_dict=sorted(freq_dict, key=lambda x:max([v[1] for v in freq_dict])-x[1])
sorted_symbols=[s[0] for s in sorted_freq_dict]

lm = ngram.LM("data/6-gram-wiki-char.lm.bz2", n=6, verbose=False)

Reading language model from data/6-gram-wiki-char.lm.bz2...
Done.


## Maximum Context Score estimation 

In [7]:

## f = cipher symbol char, 
def convert_to_bitstring(f,cipher_text) : 
    return "".join(['o' if f == t else '.' for t in cipher_text])

## bs1,bs2 = bitstrings 
def superimpose_bitstrings(bs1,bs2) : 
    
    BS1=list(bs1)
    BS2=list(bs2)
    NEW_BS=[]
    for i in range(len(BS1)) : 
        if BS2[i] == '.' and BS1[i] == '.' : 
            NEW_BS.append('.')
        else : 
            NEW_BS.append('o')
            
            
    return ''.join(NEW_BS)

## F is a list of cipher symbols 
## cipher_text is string of cipher.
def convert_to_bitstring_multiple(F,cipher_text) : 
    
    master_bs="."*len(cipher_text)
    for f in F : 
        #print('f=',f)
        #print('cipher_text=',cipher_text)
        new_bitstring=convert_to_bitstring(f,cipher_text)
        #print('new_bitstring=',new_bitstring)
        master_bs = superimpose_bitstrings(master_bs,new_bitstring)
        #print("master_bs= ",master_bs)
    return master_bs

## cipher_text is all cipher in string
## f is single cipher char
## W is list of weights (6 lengths)
## phi is HS which is [(e,f) , (e,f) ...]
def calculate_maximum_context_score(cipher_text,f,W,phi) : 
    new_phi = [i[1] for i in phi]
    new_phi.append(f)
    #print('new_phi=',new_phi)
    bitstring = convert_to_bitstring_multiple(new_phi,cipher_text)
    
    #print("bitstring= ",bitstring)
    contagious_o = re.findall(r'[o]+',bitstring)
    #print("contagiouso = ",contagious_o)
    
    contagious_lenghts = [len(i) for i in contagious_o]
    
    #print("contagious_lenghts = ",contagious_lenghts)
    N=6
    max_context=[float(len(list(filter(lambda x:x==i,contagious_lenghts)))) for i in range(1,N+1)]
    #print('max_context = ',max_context)
    term=np.multiply(W,max_context)
    #print('term= ',term)
    return sum(term)


def get_best_f_msscore(cipher_text,remaining_fs,weights,phi) : 
    MS_SCORES=[]
    for f in remaining_fs : 
        f_ms_score = calculate_maximum_context_score(cipher_text,f,weights,phi)
        MS_SCORES.append(f_ms_score)
    
    F_SCORES=dict([(i,j) for i,j in zip(MS_SCORES,remaining_fs) ])
    indmax=max(F_SCORES.keys())
    best_f=F_SCORES[indmax]
    #print(F_SCORES)
    #print(best_f)
    #print(indmax)
    return best_f

## EXAMPLE FOR TESTING

CIPHER='BURGER'
VOCAB='BUGER'
W=[1,1,1,1,2,3]
HS=[('', ''),('b','B'),('g','G')]
#calculate_maximum_context_score(CIPHER,'U',W,HS)
#get_best_f_msscore(CIPHER,['R','U'],W,HS)



## HELPER FUNCTIONS USED BY BEAM SEARCH

In [8]:

## phi_obj is list of tuples.
def satisfy_ext_limits(phi_obj,nkeep) : 
    
   # print(phi_obj)
    l = dict([(i[0],0) for i in phi_obj])
    for elem in phi_obj : 
        l[str(elem[0])]+=1
   
    n_lengths=list(filter(lambda x:x>nkeep,list(l.values())))
   
    if n_lengths == [] : 
        return True 
    else : 
        return False
    
    
## phi is [(e,f),(e,f) ... ]
## lm is language model from ngrams
## cipher is string of cipher
def score_partial_hypothesis(cipher, phi,lm) :
   
    ## reverse_phi is {f1:e1, f2:e2 ...}
    reverse_phi= dict([(i[1],i[0]) for i in phi ])
    #print('reverse_phi=',reverse_phi)
    
    f_phi_list = [i[1] for i in phi]
    #print("f_phi_list= ",f_phi_list)
    
    deciphered_tokens=[]
    arg1=[]
    
    overall_score=0
    for f in cipher : 
        
        if f in f_phi_list : 
            deciphered_tokens.append(reverse_phi[str(f)])
            arg1.append("o")
        else : 
            deciphered_tokens.append(".")  
            arg1.append(".")
    
    dt = "".join(deciphered_tokens)
    a1 = "".join(arg1)
    #print('a1 = ',a1)
    #print('dt tokens= ',dt)
    score=lm.score_bitstring(dt,a1)
    return score


def hist_prune(H,nkeep) : 

    scores = [float(i[1]) for i in H]
    scores_s = sorted(H,key=lambda x:x[1])
    #print(scores_s)
    return scores_s[-nkeep:]


def score_beam(hs,real_phi) : 
    phi = hs[0][0]
    score=0
    for p in phi : 
        if p in real_phi : 
            score=score+1
    
    correct_ratio = score/len(phi)
    return correct_ratio

def explored_space(ch,nkeep) : 
    phi=ch[0][0]
    if len(phi) > nkeep : 
        return True
    else :
        return False
    

def get_sorted_syms(x1,x2) : 
    freq_dict=[ (k,v) for k,v in zip(x2['frequencies'].keys(),x2['frequencies'].values())]
    sorted_freq_dict=sorted(freq_dict, key=lambda x:max([v[1] for v in freq_dict])-x[1])
    sorted_symbols=[s[0] for s in sorted_freq_dict]
    return sorted_symbols


#SAMPLE_PHI=[('', ''), ('e', '—'), ('e', 'º'), ('u', 'B'), ('v', 'R')]
#satisfy_ext_limits(SAMPLE_PHI,3)


#SAMPLE1=[('', ''),('b','B'),('r','R'),('g','G'),('e','E')]
#CIPHER="BURGER"
#LM=lm
#score_partial_hypothesis(CIPHER,SAMPLE1,LM)


#SAMPLE=[([('', ''), ('e', 'O'), ('h', 'T')], -32.71637948), ([('', ''), ('e', 'O'), ('s', 'T')], -21.396480099999998), ([('', ''), ('e', 'O'), ('o', 'T')], -39.44501403), ([('', ''), ('e', 'O'), ('j', 'T')], -33.47798432999999), ([('', ''), ('e', 'O'), ('d', 'T')], -20.987173), ([('', ''), ('e', 'O'), ('v', 'T')], -30.673667710000004), ([('', ''), ('e', 'O'), ('g', 'T')], -29.781602661000004), ([('', ''), ('e', 'O'), ('f', 'T')], -28.604963550999997), ([('', ''), ('e', 'O'), ('a', 'T')], -29.75088907), ([('', ''), ('e', 'O'), ('r', 'T')], -24.553558436), ([('', ''), ('e', 'O'), ('x', 'T')], -32.67641992), ([('', ''), ('e', 'O'), ('m', 'T')], -30.1644232), ([('', ''), ('e', 'O'), ('t', 'T')], -29.95256448), ([('', ''), ('e', 'O'), ('b', 'T')], -26.972342800000003), ([('', ''), ('e', 'O'), ('u', 'T')], -34.10374856), ([('', ''), ('e', 'O'), ('c', 'T')], -33.261454670000006), ([('', ''), ('e', 'O'), ('p', 'T')], -29.9686529), ([('', ''), ('e', 'O'), ('q', 'T')], -38.0978153), ([('', ''), ('e', 'O'), ('w', 'T')], -31.314459720000002), ([('', ''), ('e', 'O'), ('k', 'T')], -29.489249459999996), ([('', ''), ('e', 'O'), ('n', 'T')], -27.4332253), ([('', ''), ('e', 'O'), ('z', 'T')], -31.70670717), ([('', ''), ('e', 'O'), ('i', 'T')], -30.30622924), ([('', ''), ('e', 'O'), ('y', 'T')], -32.38038479), ([('', ''), ('e', 'O'), ('l', 'T')], -26.465809439999997)]
#hist_prune(SAMPLE,1)

#SAMPLE= [([('', ''), ('x', 'E'), ('i', 'T'), ('a', 'N'), ('s', 'A'), ('m', 'O'), ('e', 'R'), ('j', 'U'), ('n', 'B'), ('t', 'D'), ('o', 'P'), ('r', 'I'), ('u', 'H'), ('g', 'L'), ('f', 'S'), ('q', 'Y'), ('h', 'X'), ('y', 'G'), ('b', 'W'), ('p', 'V'), ('d', 'M')], -291.5569235510001)]

#CIPHER="".join(cipher_desc['content'])
#W=[1,1,1,1,2,3]
#print(sort_by_new_extension_order(CIPHER,cipher_desc['vocab'],W))

    
#W=[1,1,1,1,2,3]
#F='R'
#CIPHER='BURGERISRREFRRRRRINED'
#calculate_maximum_context_score(CIPHER,F,W)



In [9]:
# Helper func for debugging
# Change the 'isverbose' to True to print
def printifverbose(text, isverbose=False):
    if isverbose:
        print(text)
        
        


## BEAM SEARCH ALGORITHM 

In [10]:
## ext_order is cipher tokens list
## ext_limits is number
## Vf is unique cipher tokens list 
## Ve is unique english tokens list (a-z)
## nkeep is number
## cipher_text is string of all the cipher 
## msscore_enabled is bool

def beam_search(ext_order, ext_limits,Vf,Ve,nkeep,cipher_text,msscore_enabled):
    # FOR 'BURGER' EXAMPLE:
    # ext_order: ['R', 'B', 'U', 'G', 'E']
    # Vf: ['U', 'E', 'B', 'G', 'R']
    # cipher_text: "burger"
    
    printifverbose(str("==startbeamsearch==").upper())
   
    Hs = []
    Ht = []
    # Hs and Ht will be of format '[phi, score]' 
    # which is '[list, float]'
    # [([('', '')], 0)]
    cardinality = 0
    Hs.append(([('','')],0))
    #print(Hs[0])
    new_phi=[]
    
    W=[1,1,1,1,2,3]
    
    
    remaining_f=[]
    for i in Vf : 
        remaining_f.append(i)
        
    current_hs=[]
    printifverbose("len(Vf): " + str(len(Vf)))
    
    
    while cardinality < len(Vf):  #line5
        printifverbose("\t--beginwhile--")
        
        
        if msscore_enabled : 
            f = get_best_f_msscore("".join(cipher_desc['content']),remaining_f,W,Hs) ## MAXIMUM CONTEXT IMPLEMENTATION
        else : 
            f = ext_order[cardinality]  # ORIGINAL IMPLEMENTATION
    
        
        printifverbose("\tCurrent cipher character (f): " + f + "\n")

        # Hs is in:
        # [([('', '')], 0)]
        
        current_hs=[]
        for h in Hs:  #line7a
            
            
            
            printifverbose("\t\t--beginOuterloop--")
            phi=h[0]  #line7b
            printifverbose("\t\tlen(Ve):" + str(len(Ve)) + "\n")
            
            
            for e in Ve:  #line8
                printifverbose("\t\t\t--beginInnerloop--")
                
                printifverbose("\t\t\tcurrent (e) --> '" + e + "'")
                
                new_eandf=(e,f)  #line9a
                printifverbose("\t\t\tcurrent (e,f) --> ('" + e + "','" + f + "')")
                
                new_phi=[]
                for p in phi : 
                    new_phi.append(p)
                 
                new_phi.append(new_eandf)
                printifverbose("\t\t\tϕ' = ϕ ∪ {(e,f)}")
                printifverbose("\t\t\t--> " + str(new_phi))
                
                # SCORE
                if satisfy_ext_limits(new_phi,ext_limits):  #line10
                    SCORE=score_partial_hypothesis(cipher_text,new_phi,lm)  #line11a
                    ht_entry=(new_phi,SCORE)  #line11b
                    Ht.append((ht_entry))  #line 11c
                    
                printifverbose("\t\t\t(ϕ', SCORE(ϕ'))")
                printifverbose("\t\t\t--> " + str(ht_entry) + "   ##Add to Ht")
                printifverbose("\t\t\t--endInnerloop--\n")

            ## INNER LOOP ENDS
   
            printifverbose("\t\tHt --> " + str(Ht)) # + "\n")
            printifverbose("\t\t--endOuterloop--\n")
        
            prune = hist_prune(Ht,nkeep)
            for p in prune : 
                current_hs.append(p)
            Ht=[]
        
        cardinality = cardinality + 1  #line13
        
        printifverbose("\tHt after prunning --> " + str(Ht)) # + "\n")
        printifverbose("\n\tHs = Ht\n\tHs --> " + str(Ht)) # + "\n")
        
        Hs = current_hs
        Ht=[]  #line15
    
        printifverbose("\t--endwhile--" + "\n")
        remaining_f.remove(f)
        
        
    printifverbose("==endbeamsearch==" + "\n")
    return Hs  #WINNER(Hs)

## TOY EXAMPLE TO TEST BEAM SEARCH

In [16]:
## TESTING BEAM SEARCH ON SIMPLE 1:1 SUBSITUTION CIPHER

sample_text="thisismine"
cipher_text = sample_text.upper() ## BURGER is the CIPHER


## x2 is plaintext  get_statistics() object
## x1 is ciphertext get_statistics() object

def get_sorted_syms(x1,x2) : 
    freq_dict=[ (k,v) for k,v in zip(x2['frequencies'].keys(),x2['frequencies'].values())]
    sorted_freq_dict=sorted(freq_dict, key=lambda x:max([v[1] for v in freq_dict])-x[1])
    sorted_symbols=[s[0] for s in sorted_freq_dict]
    return sorted_symbols



cipher_desc = get_statistics(cipher_text,cipher=True)

ss=get_sorted_syms(plaintxt_desc,cipher_desc)
W=[1.0,1.0,1.0,1.0,2,3]

ALPHA1="abcdefghijklmnopqrstuvwxyz"
ALPHA2="ABCDEFGHIJKLMNOPQRSTUVWXYZ" ## CIPHER
REAL_PHI=[(a,b) for a,b in zip(ALPHA1,ALPHA2)]

EXT_ORDER=ss
EXT_LIMIT=1
KEEPS=7
VF=cipher_desc['vocab'] ## LIST OF CIPHER TOKENS
VE=plaintxt_desc['vocab']


final_hs=beam_search(EXT_ORDER,EXT_LIMIT,VF,VE,KEEPS,cipher_text,msscore_enabled=False)
print("final_hs = ",final_hs)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## FOR TOY EXAMPLE, FINDING ACCURACY OF ALGORITHM (NGRAM SCORE VS CIPHER-KEY SCORE)

In [17]:
#ind=np.argmin([f[1] for f in final_hs] )
#final_hs


def score_beam(hs,real_phi) : 
    
    set1=set(hs)
    set2=set(real_phi)
    common=set1.intersection(set2)
    return len(common)/len(set1)



phi=[f[0] for f in final_hs]
REAL_SCORES=[]
for p in phi : 
    REAL_SCORES.append(score_beam(p,REAL_PHI))
    
    
ind1=np.argmax([f[1] for f in final_hs] )
ind2=np.argmax(REAL_SCORES)
    
print("BEST RULES ACCORDING TO NGRAM SCORING = ",final_hs[ind1])
print("BEST ACTUAL SCORE (COMPARING WITH REAL CIPHER KEY) = ",final_hs[ind2])


## here same for both. This may mean that our score function is working correctly.

BEST RULES ACCORDING TO NGRAM SCORING =  ([('', ''), ('e', 'I'), ('m', 'S'), ('t', 'T'), ('h', 'H'), ('b', 'M'), ('r', 'N'), ('s', 'E')], -6.58936822386)
BEST ACTUAL SCORE (COMPARING WITH REAL CIPHER KEY) =  ([('', ''), ('i', 'I'), ('s', 'S'), ('t', 'T'), ('h', 'H'), ('m', 'M'), ('n', 'N'), ('e', 'E')], -10.32367101)


In [878]:
[f[1] for f in final_hs] 

[-20.52483247,
 -20.469566370000003,
 -20.340698350000004,
 -19.97365257,
 -19.651993770000004,
 -20.754139990000002,
 -20.44615419,
 -20.39088809,
 -20.262020070000002,
 -19.573315490000002,
 -20.0895483,
 -19.7815625,
 -19.7262964,
 -19.2303826,
 -18.9087238,
 -18.4316202,
 -18.1236344,
 -18.0683683,
 -17.93950028,
 -17.5724545,
 -17.97114064,
 -17.91587454,
 -17.787006520000002,
 -17.41996074,
 -17.098301940000002,
 -21.439509799999996,
 -21.3842437,
 -21.25537568,
 -20.8883299,
 -20.5666711,
 -21.510125399999996,
 -21.2021396,
 -21.146873499999995,
 -20.650959699999998,
 -20.3293009,
 -20.76373325,
 -20.70846715,
 -20.579599130000002,
 -20.21255335,
 -19.890894550000002,
 -20.1222502,
 -19.8142644,
 -19.63013028,
 -19.263084499999998,
 -18.9414257,
 -19.179814620000002,
 -18.87182882,
 -18.81656272,
 -18.6876947,
 -18.32064892,
 -21.4632212,
 -21.1552354,
 -21.099969299999998,
 -20.6040555,
 -20.2823967,
 -21.595881659999996,
 -21.194817559999997,
 -20.886831759999996,
 -20.8315656

In [13]:
REAL_SCORES

[0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.25,
 0.125,
 0.125,
 0.125,
 0.125,
 0.25,
 0.125,
 0.125,
 0.125,
 0.25,
 0.125,
 0.125,
 0.125,
 0.125,
 0.25,
 0.125,
 0.125,
 0.125,
 0.125,
 0.25,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.25,
 0.125,
 0.125,
 0.125,
 0.125,
 0.25,
 0.125,
 0.125,
 0.125,
 0.125,
 0.25,
 0.125,
 0.125,
 0.125,
 0.125,
 0.25,
 0.125,
 0.125,
 0.125,
 0.125,
 0.25,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.0,
 0.0